In [5]:
# Import the chit fund analyzer module
from chit_fund_analyzer import ChitFundConfig, ChitFundAnalyzer, ScenarioAnalyzer
from chit_fund_analyzer.utils import ChitFundFormatter, format_currency, format_percentage
from decimal import Decimal
import pandas as pd
import numpy as np

## 1. Basic Configuration and Validation

Let's start by creating a chit fund configuration with automatic validation:

In [6]:
# Create a chit fund configuration with validation
try:
    config = ChitFundConfig(
        total_installments=14,
        current_installment_number=5,
        full_chit_value=Decimal('700000'),
        chit_frequency_per_year=2,  # Bi-annual
        previous_installments=[
            Decimal('42000'), 
            Decimal('40000'), 
            Decimal('40000'), 
            Decimal('43000')
        ],
        bid_amount=Decimal('100000')
    )
    print("✅ Configuration created successfully!")
    print(f"Winner installment amount: {format_currency(config.get_winner_installment_amount())}")
    
except Exception as e:
    print(f"❌ Configuration error: {e}")

✅ Configuration created successfully!
Winner installment amount: ₹50,000.00


## 2. Input Validation Examples

The module includes comprehensive input validation. Let's see some examples:

In [7]:
# Example 1: Invalid current installment number
try:
    invalid_config = ChitFundConfig(
        total_installments=10,
        current_installment_number=15,  # Greater than total!
        full_chit_value=Decimal('500000'),
        chit_frequency_per_year=12,
        previous_installments=[],
        bid_amount=Decimal('50000')
    )
except Exception as e:
    print(f"✅ Caught validation error: {e}")

# Example 2: Mismatched previous installments count
try:
    invalid_config = ChitFundConfig(
        total_installments=10,
        current_installment_number=5,
        full_chit_value=Decimal('500000'),
        chit_frequency_per_year=12,
        previous_installments=[Decimal('30000'), Decimal('30000')],  # Should be 4 items for installment 5
        bid_amount=Decimal('50000')
    )
except Exception as e:
    print(f"✅ Caught validation error: {e}")

# Example 3: Bid amount too high
try:
    invalid_config = ChitFundConfig(
        total_installments=10,
        current_installment_number=3,
        full_chit_value=Decimal('500000'),
        chit_frequency_per_year=12,
        previous_installments=[Decimal('30000'), Decimal('30000')],
        bid_amount=Decimal('500000')  # Equal to full chit value!
    )
except Exception as e:
    print(f"✅ Caught validation error: {e}")

✅ Caught validation error: 1 validation error for ChitFundConfig
current_installment_number
  Value error, Current installment (15) cannot exceed total installments (10) [type=value_error, input_value=15, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
✅ Caught validation error: 1 validation error for ChitFundConfig
previous_installments
  Value error, Number of previous installments (2) should be 4 for current installment number 5 [type=value_error, input_value=[Decimal('30000'), Decimal('30000')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
✅ Caught validation error: 1 validation error for ChitFundConfig
bid_amount
  Value error, Bid amount (500000) cannot be >= full chit value (500000) [type=value_error, input_value=Decimal('500000'), input_type=Decimal]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


## 3. Basic Analysis

Now let's perform the chit fund analysis using our validated configuration:

In [8]:
# Create analyzer and perform analysis
analyzer = ChitFundAnalyzer(config)
result = analyzer.analyze()

# Display formatted results
print(ChitFundFormatter.format_analysis_summary(result))

=== CHIT FUND ANALYSIS SUMMARY ===
Total Installments: 14
Current Installment: 5
Full Chit Value: ₹700,000.00
Bid Amount: ₹100,000.00
Prize Amount: ₹550,000.00
Winner Installment: ₹50,000.00

=== RETURNS ===
Annual IRR: 8.82%
Period IRR: 4.3155%
Payment Frequency: 2 times per year


In [9]:
# Display cashflows for better understanding
print("\n=== CASHFLOW BREAKDOWN ===")
print(f"Number of cashflows: {len(result.cashflows)}")
print("\nCashflow details:")
for i, cf in enumerate(result.cashflows, 1):
    flow_type = "INFLOW" if cf > 0 else "OUTFLOW"
    print(f"Period {i:2d}: {format_currency(cf):>12} ({flow_type})")

print(f"\nNet cashflow: {format_currency(sum(result.cashflows))}")


=== CASHFLOW BREAKDOWN ===
Number of cashflows: 14

Cashflow details:
Period  1:  ₹-42,000.00 (OUTFLOW)
Period  2:  ₹-40,000.00 (OUTFLOW)
Period  3:  ₹-40,000.00 (OUTFLOW)
Period  4:  ₹-43,000.00 (OUTFLOW)
Period  5:  ₹550,000.00 (INFLOW)
Period  6:  ₹-50,000.00 (OUTFLOW)
Period  7:  ₹-50,000.00 (OUTFLOW)
Period  8:  ₹-50,000.00 (OUTFLOW)
Period  9:  ₹-50,000.00 (OUTFLOW)
Period 10:  ₹-50,000.00 (OUTFLOW)
Period 11:  ₹-50,000.00 (OUTFLOW)
Period 12:  ₹-50,000.00 (OUTFLOW)
Period 13:  ₹-50,000.00 (OUTFLOW)
Period 14:  ₹-50,000.00 (OUTFLOW)

Net cashflow: ₹-65,000.00


## 4. Scenario Analysis

Let's analyze multiple bid scenarios to understand the impact of different bid amounts:

In [10]:
# Create scenario analyzer
scenario_analyzer = ScenarioAnalyzer(config)

# Define bid amounts to analyze
bid_amounts = [Decimal(str(amount)) for amount in [80000, 90000, 100000, 110000, 120000, 130000]]

# Perform scenario analysis
scenarios = scenario_analyzer.analyze_bid_scenarios(bid_amounts)

# Create and display DataFrame
scenario_df = scenario_analyzer.create_scenario_dataframe(scenarios)
print("=== SCENARIO ANALYSIS RESULTS ===")
print(scenario_df.to_string(index=False))

=== SCENARIO ANALYSIS RESULTS ===
Bid Amount Prize Amount  Annual IRR Annual IRR (%) Bid Amount (Formatted) Prize Amount (Formatted)
     80000       570000    0.056488          5.65%             ₹80,000.00              ₹570,000.00
     90000       560000    0.071672          7.17%             ₹90,000.00              ₹560,000.00
    100000       550000    0.088172          8.82%            ₹100,000.00              ₹550,000.00
    110000       540000    0.106253         10.63%            ₹110,000.00              ₹540,000.00
    120000       530000    0.126272         12.63%            ₹120,000.00              ₹530,000.00
    130000       520000    0.148740         14.87%            ₹130,000.00              ₹520,000.00


## 5. Frequency Comparison

Let's compare how different payment frequencies affect the returns for the same bid amount:

## 6. Advanced Configuration with Custom Winner Installment

Sometimes the winner might negotiate a different installment amount. Let's see how this affects the analysis:

In [13]:
# Compare different payment frequencies
frequency_comparison = scenario_analyzer.compare_frequencies(
    bid_amount=Decimal('100000'),
    frequencies=[1, 2, 4, 6, 12]  # Annual, bi-annual, quarterly, bi-monthly, monthly
)

print("=== FREQUENCY COMPARISON ===")
print(frequency_comparison.to_string(index=False))

=== FREQUENCY COMPARISON ===
 Frequency per Year Period Type Bid Amount Prize Amount  Annual IRR Annual IRR (%)  Period IRR Period IRR (%)
                  1      Custom     100000       550000    0.043155          4.32%    0.043155        4.3155%
                  2 Half-Yearly     100000       550000    0.088172          8.82%    0.043155        4.3155%
                  4   Quarterly     100000       550000    0.184119         18.41%    0.043155        4.3155%
                  6      Custom     100000       550000    0.288526         28.85%    0.043155        4.3155%
                 12     Monthly     100000       550000    0.660298         66.03%    0.043155        4.3155%


## 7. Error Handling and Edge Cases

The module provides robust error handling. Let's demonstrate some edge cases:

In [14]:
# Create configuration with custom winner installment
custom_config = ChitFundConfig(
    total_installments=14,
    current_installment_number=5,
    full_chit_value=Decimal('700000'),
    chit_frequency_per_year=2,
    previous_installments=[Decimal('42000'), Decimal('40000'), Decimal('40000'), Decimal('43000')],
    bid_amount=Decimal('100000'),
    winner_installment_amount=Decimal('45000')  # Custom amount instead of default 50K
)

# Analyze with custom configuration
custom_analyzer = ChitFundAnalyzer(custom_config)
custom_result = custom_analyzer.analyze()

print("=== ANALYSIS WITH CUSTOM WINNER INSTALLMENT ===")
print(ChitFundFormatter.format_analysis_summary(custom_result))

# Compare with original
print(f"\n=== COMPARISON ===")
print(f"Default installment IRR: {format_percentage(result.annual_irr)}")
print(f"Custom installment IRR:  {format_percentage(custom_result.annual_irr)}")
print(f"Difference: {format_percentage(custom_result.annual_irr - result.annual_irr)} (better)" if custom_result.annual_irr > result.annual_irr else f"Difference: {format_percentage(result.annual_irr - custom_result.annual_irr)} (worse)")

=== ANALYSIS WITH CUSTOM WINNER INSTALLMENT ===
=== CHIT FUND ANALYSIS SUMMARY ===
Total Installments: 14
Current Installment: 5
Full Chit Value: ₹700,000.00
Bid Amount: ₹100,000.00
Prize Amount: ₹555,000.00
Winner Installment: ₹45,000.00

=== RETURNS ===
Annual IRR: 1.96%
Period IRR: 0.9742%
Payment Frequency: 2 times per year

=== COMPARISON ===
Default installment IRR: 8.82%
Custom installment IRR:  1.96%
Difference: 6.86% (worse)


## 8. Data Export and Reporting

The module provides utilities for exporting analysis results:

In [ ]:
# Example: Bid amount too high leading to negative prize
try:
    problematic_config = ChitFundConfig(
        total_installments=10,
        current_installment_number=3,
        full_chit_value=Decimal('200000'),
        chit_frequency_per_year=12,
        previous_installments=[Decimal('15000'), Decimal('15000')],
        bid_amount=Decimal('180000')  # Very high bid
    )
    
    problematic_analyzer = ChitFundAnalyzer(problematic_config)
    problematic_result = problematic_analyzer.analyze()
    
except Exception as e:
    print(f"✅ Caught analysis error: {e}")
    print("The module prevents invalid scenarios that would result in negative prize amounts.")

## 9. Summary and Best Practices

This demonstration showcases the key features of the `chit_fund_analyzer` module:

In [ ]:
# Create comprehensive analysis report
report_data = {
    'Base Analysis': pd.DataFrame([analyzer.format_results(result)]),
    'Scenario Analysis': scenario_df,
    'Frequency Comparison': frequency_comparison
}

# Display summary
print("=== EXPORT READY DATA ===")
for sheet_name, df in report_data.items():
    print(f"\n{sheet_name}:")
    print(f"  - {len(df)} rows")
    print(f"  - Columns: {', '.join(df.columns)}")

print("\n✅ Ready for Excel export using utils.export_to_excel()")

## 9. Summary and Best Practices

This demonstration showcases the key features of the `chit_fund_analyzer` module:

### ✅ **What We've Accomplished:**
- **Type Safety**: All inputs are validated using Pydantic models
- **Clean Architecture**: Separated concerns with dedicated classes
- **Comprehensive Validation**: Multiple layers of input validation
- **Extensible Design**: Easy to add new analysis methods
- **Error Handling**: Graceful handling of edge cases and invalid inputs
- **Modern Python**: Uses modern Python features and best practices

### 🚀 **Extension Points:**
- Add new validation rules in `models.py`
- Extend analysis methods in `analyzer.py`
- Add new scenario types in `scenario.py`
- Implement additional utilities in `utils.py`
- Create custom formatters for different output formats

### 📊 **Usage Recommendations:**
1. Always use the validation features - they prevent calculation errors
2. Leverage scenario analysis for decision-making by comparing min/max ranges
3. Use frequency comparison to understand payment impact
4. Export results for stakeholder reporting
5. Handle exceptions gracefully in production code